In [19]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import label_binarize

In [20]:
df = pd.read_csv('../data/aden.csv',sep=';')

In [21]:
max_quinzaine=24
max_area_lat=45
max_area_lon=53
number_grid=(max_area_lat+1)*(max_area_lon+1)
print number_grid

2484


In [22]:
df.columns

Index([u'Unnamed: 0', u'Unnamed: 0.1', u'Unnamed: 0.1', u'Annee (0)',
       u'jour (0)', u'mois (0)', u'dist cote (nq)', u'zone', u'lat (0)',
       u'long(0)', u'type(0)', u'pavillon', u'statut de l'attaque(0)',
       u'type d'attaque(0)', u'Nombre P  Min', u'Nombre P  max',
       u'nombre embarcation', u'Vol a bord', u' Kidnapping', u'blessees',
       u' meurtre', u'Knives ', u'RPG', u'descriptif', u'latitude',
       u'longitude', u'quinzaine_week', u'newDescriptif', u'Guns', u'mousson',
       u'distance au port', u'distance_shorline', u'lat_area', u'lon_area'],
      dtype='object')

# Version 1

In [5]:
df['area'] = df.apply(lambda row: str(row['lat_area'])+'_'+str(row['lon_area']), axis=1)

## Creation d'un data frame vide

In [6]:
dfAgregateTotal=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","quinzaine_week","area"] ).size()}).reset_index()

In [7]:
specifecArea=pd.DataFrame(dfAgregateTotal['area'].str.split('_',1).tolist(),columns = ['lat_area','lon_area'])
dfAgregateTotal=pd.concat([dfAgregateTotal, specifecArea], axis=1)
print dfAgregateTotal.head()

   Annee (0)  quinzaine_week   area  count lat_area lon_area
0       2008               3  24_22      2       24       22
1       2008               3  35_41      1       35       41
2       2008               3    3_9      1        3        9
3       2008               3    5_9      1        5        9
4       2008               5  15_29      1       15       29


In [8]:
uniqueYears=dfAgregateTotal['Annee (0)'].unique()
Colyears=np.repeat(uniqueYears, 24*(max_area_lon+1)*(max_area_lat+1))

TwoweekRepeat=np.repeat(np.linspace(1, max_quinzaine, num=max_quinzaine), (max_area_lon+1)*(max_area_lat+1))
ColTwoweek=np.tile(TwoweekRepeat, len(uniqueYears))

latRepeat=np.linspace(0, max_area_lat,num=max_area_lat+1)
ColLatArea=np.repeat(latRepeat, max_area_lon+1)
ColLatArea=np.tile(ColLatArea, max_quinzaine)
ColLatArea=np.tile(ColLatArea, len(uniqueYears))

#print ColLatArea

lonRepeat=np.tile(np.linspace(0, max_area_lon, num=max_area_lon+1),max_area_lat+1)
ColLonArea=np.tile(lonRepeat, max_quinzaine)
ColLonArea=np.tile(ColLonArea, len(uniqueYears))

print ColLonArea

count=np.repeat(0, len(Colyears))

print len(Colyears)
print len(ColTwoweek)
print len(ColLatArea)
print len(ColLonArea)


[  0.   1.   2. ...,  51.  52.  53.]
417312
417312
417312
417312


## Fill the matrice

In [9]:
#columns = ['Years','2_week', 'lat_area','lon_area','count']
#matriceML=pd.DataFrame(columns=columns)
#matriceML = matriceML.fillna(0)

dic = {'Years': Colyears, 
     '2_week': ColTwoweek, 
     'lat_area': ColLatArea,
     'lon_area': ColLonArea,
     'count': count
      }

dfFinal= pd.DataFrame(dic)

dfFinal['lat_area']=dfFinal['lat_area'].astype(int).astype('str')
dfFinal['lon_area']=dfFinal['lon_area'].astype(int).astype('str')

print dfFinal.head()

   2_week  Years  count lat_area lon_area
0       1   2008      0        0        0
1       1   2008      0        0        1
2       1   2008      0        0        2
3       1   2008      0        0        3
4       1   2008      0        0        4


In [10]:
for index, row in dfAgregateTotal.iterrows():
    if index%100==0:
        print index
    #print row['Annee (0)'], row['quinzaine_week'], row['count'] ,  row['lat_area'] , row['lon_area'] 
    idx=dfFinal[(dfFinal['Years']==row['Annee (0)']) & (dfFinal['lat_area']==row['lat_area']) & (dfFinal['lon_area']==row['lon_area'] )& (dfFinal['2_week']==row['quinzaine_week'])]
    #print 'index : '+str(idx.index.values)
    #print 'count : '+str(row['count'])
    dfFinal.set_value(idx.index.values, 'count',row['count'])

0
100
200
300
400
500
600
700


In [46]:
dfFinal['count'].sum()

909

## Write matrice in a file

In [47]:
dfFinal.to_csv("../data/adenMatrix.csv",sep=';', encoding='utf-8', index=False)

#  

# Version 2

In [36]:
dfAgregateTotal2=pd.DataFrame({'count' : df.groupby( [ "Annee (0)","quinzaine_week","type d'attaque(0)"] ).size()}).reset_index()
dfAgregateTotal2.head()

,Annee (0),quinzaine_week,type d'attaque(0),count
0,2008,3,Boarded,3
1,2008,3,Fired upon,1
2,2008,3,Hijacked,1
3,2008,5,Attempted,1
4,2008,5,Boarded,2


In [37]:
unique_years=dfAgregateTotal2['Annee (0)'].unique()

unique_type=dfAgregateTotal2['type d\'attaque(0)'].unique()

print unique_years
print unique_type

[2008 2009 2010 2011 2012 2013 2014]
['Boarded' 'Fired upon' 'Hijacked' 'Attempted']


In [38]:
col_years=np.repeat(unique_years, max_quinzaine*4)
print len(col_years)

two_week_repeat=np.linspace(1, max_quinzaine, num=max_quinzaine)
print np.repeat(two_week_repeat,4)
col_two_week=np.tile(np.repeat(two_week_repeat,4), len(uniqueYears))
print len(col_two_week)

col_type=np.tile(unique_type, max_quinzaine*len(unique_years))
print len(col_type)



col_count=np.repeat(0, len(col_two_week))


dic2 = {'Years': col_years, 
     '2_week': col_two_week, 
     'count': col_count,
     'type_attack': col_type
      }

dfFinal_2= pd.DataFrame(dic2)




#for i in range(0,max_area_lat+1):
#    colName='lat_area_'+str(i)
#    #print colName
#    dfFinal_2[colName]=0

#for i in range(0,max_area_lon+1):
#    colName='lon_area_'+str(i)
#    #print colName
#    dfFinal_2[colName]=0
    


672
[  1.   1.   1.   1.   2.   2.   2.   2.   3.   3.   3.   3.   4.   4.   4.
   4.   5.   5.   5.   5.   6.   6.   6.   6.   7.   7.   7.   7.   8.   8.
   8.   8.   9.   9.   9.   9.  10.  10.  10.  10.  11.  11.  11.  11.  12.
  12.  12.  12.  13.  13.  13.  13.  14.  14.  14.  14.  15.  15.  15.  15.
  16.  16.  16.  16.  17.  17.  17.  17.  18.  18.  18.  18.  19.  19.  19.
  19.  20.  20.  20.  20.  21.  21.  21.  21.  22.  22.  22.  22.  23.  23.
  23.  23.  24.  24.  24.  24.]
672
672


## Matrix empty

In [39]:
dfFinal_2.head()

,2_week,Years,count,type_attack
0,1,2008,0,Boarded
1,1,2008,0,Fired upon
2,1,2008,0,Hijacked
3,1,2008,0,Attempted
4,2,2008,0,Boarded


# Fill matrice

In [40]:
for index, row in dfAgregateTotal2.iterrows():

    print 'Years '+str(row['Annee (0)']),'2_week : '+str( row['quinzaine_week']),'count : '+str(row['count']),'type_attack : '+str(row['type d\'attaque(0)'])

    idx=dfFinal_2[(dfFinal_2['Years']==row['Annee (0)']) & (dfFinal_2['2_week']==row['quinzaine_week'])& (dfFinal_2['type_attack']==row['type d\'attaque(0)'])]
    print 'index : '+str(idx.index.values)
    print 'count : '+str(row['count'])
    dfFinal_2.set_value(idx.index.values, 'count',row['count'])

Years 2008 2_week : 3.0 count : 3 type_attack : Boarded
index : [8]
count : 3
Years 2008 2_week : 3.0 count : 1 type_attack : Fired upon
index : [9]
count : 1
Years 2008 2_week : 3.0 count : 1 type_attack : Hijacked
index : [10]
count : 1
Years 2008 2_week : 5.0 count : 1 type_attack : Attempted
index : [19]
count : 1
Years 2008 2_week : 5.0 count : 2 type_attack : Boarded
index : [16]
count : 2
Years 2008 2_week : 6.0 count : 1 type_attack : Attempted
index : [23]
count : 1
Years 2008 2_week : 6.0 count : 3 type_attack : Boarded
index : [20]
count : 3
Years 2008 2_week : 6.0 count : 1 type_attack : Fired upon
index : [21]
count : 1
Years 2008 2_week : 7.0 count : 1 type_attack : Boarded
index : [24]
count : 1
Years 2008 2_week : 7.0 count : 1 type_attack : Fired upon
index : [25]
count : 1
Years 2008 2_week : 7.0 count : 2 type_attack : Hijacked
index : [26]
count : 2
Years 2008 2_week : 8.0 count : 2 type_attack : Attempted
index : [31]
count : 2
Years 2008 2_week : 8.0 count : 1 typ

In [43]:
binary=label_binarize(col_type, classes=unique_type)
dfFinal_2[unique_type[0]]=binary[:,0]
dfFinal_2[unique_type[1]]=binary[:,1]
dfFinal_2[unique_type[2]]=binary[:,2]
dfFinal_2[unique_type[3]]=binary[:,3]
print dfFinal_2

     2_week  Years  count type_attack  Boarded  Fired upon  Hijacked  \
0         1   2008      0     Boarded        1           0         0   
1         1   2008      0  Fired upon        0           1         0   
2         1   2008      0    Hijacked        0           0         1   
3         1   2008      0   Attempted        0           0         0   
4         2   2008      0     Boarded        1           0         0   
5         2   2008      0  Fired upon        0           1         0   
6         2   2008      0    Hijacked        0           0         1   
7         2   2008      0   Attempted        0           0         0   
8         3   2008      3     Boarded        1           0         0   
9         3   2008      1  Fired upon        0           1         0   
10        3   2008      1    Hijacked        0           0         1   
11        3   2008      0   Attempted        0           0         0   
12        4   2008      0     Boarded        1           0      

In [44]:
dfFinal_2.to_csv("../data/adenMatrix.csv",sep=';', encoding='utf-8', index=False)